## Auto Scaling en DynamoDB

Muchas cargas de trabajo de base de datos son de naturaleza cíclica, mientras que otras son difíciles de predecir con antelación. Por ejemplo, tomemos una aplicación de redes sociales en la que la mayoría de los usuarios están activos en el horario diurno. La base de datos debe satisfacer los requisitos de la actividad diurna, pero no se requieren los mismos niveles de rendimiento por la noche. Otro ejemplo: tomemos una nueva aplicación de juegos para móviles cuya adopción está siendo inesperadamente rápida. Si el juego adquiere demasiada popularidad, podría superar los recursos disponibles en la base de datos, lo que daría lugar a un rendimiento lento y a clientes descontentos. Estos tipos de cargas de trabajo suelen requerir intervención manual para escalar los recursos de la base de datos en sentido ascendente o descendente en respuesta a las variaciones en los niveles de uso.

El escalado automático de Amazon DynamoDB usa el servicio Auto Scaling de aplicaciones de AWS para ajustar de manera dinámica y automática la capacidad de rendimiento aprovisionada en respuesta a los patrones de tráfico reales. Esto permite a una tabla o índice secundario global (GSI) incrementar su capacidad de lectura y escritura aprovisionada para abastecer incrementos repentinos del tráfico sin limitaciones. Cuando la carga de trabajo disminuye, el Auto Scaling de aplicaciones puede reducir el rendimiento para evitar que tenga que pagar por una capacidad aprovisionada que no se utiliza.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
dt=DynamoTable()

In [2]:
dt=DynamoTable()
try:
    dt.select_table('TestTable')
    print(dt)
except:
    dt.create_table(
        table_name='TestTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
        provisioned=True,
        read_capacity=5,
        write_capacity=5
    )

Table created successfully!


El escalado automático de Amazon DynamoDB utiliza el servicio AWS Application Auto Scaling para ajustar dinámicamente la capacidad de desempeño provisionada en su nombre, en respuesta a los patrones de tráfico reales. Esto permite que una tabla o un índice secundario global aumente su capacidad de lectura y escritura provisionada para gestionar aumentos repentinos del tráfico, sin estrangulamiento. Cuando la carga de trabajo disminuye, Application Auto Scaling reduce el rendimiento para que no tenga que pagar por la capacidad provisionada no utilizada.

Con Application Auto Scaling, se crea una política de escalado para una tabla o un índice secundario global. La política de escalado especifica si desea escalar la capacidad de lectura o la capacidad de escritura (o ambas), así como la configuración mínima y máxima de la unidad de capacidad provisionada para la tabla o el índice. La política de escalado también contiene un objetivo de utilización, es decir, el porcentaje de rendimiento provisionado consumido en un momento dado. El escalado automático de aplicaciones utiliza un algoritmo de seguimiento de objetivos para ajustar el rendimiento provisionado de la tabla (o índice) al alza o a la baja en respuesta a las cargas de trabajo reales, de modo que la utilización real de la capacidad se mantenga en o cerca de su utilización objetivo. Puede establecer los valores de utilización objetivo de autoescalado entre el 20 y el 90 por ciento de su capacidad de lectura y escritura.

### Registrar un objetivo escalable

Ahora, vamos a registrar la capacidad de escritura de la tabla como objetivo escalable con Auto Scaling de aplicaciones. Esto permite que Auto Scaling de aplicaciones ajuste la capacidad de escritura aprovisionada para TestTable, pero solo dentro del rango de entre 5 y 10 unidades de capacidad.

**NOTA:** La función Auto Scaling de DynamoDB requiere la presencia de un rol *AWSServiceRoleForApplicationAutoScaling_DynamoDBTable* que lleve a cabo las acciones de escalado automático en su nombre. Este rol se crea automáticamente para usted.

In [8]:
client = boto3.client('application-autoscaling')

In [9]:
service_namespace = "dynamodb"
resource_id = "table/TestTable"
scalable_dimension = "dynamodb:table:WriteCapacityUnits"
min_capacity = 5
max_capacity = 10

In [10]:
try:
    response = client.register_scalable_target(
        ServiceNamespace=service_namespace,
        ResourceId=resource_id,
        ScalableDimension=scalable_dimension,
        MinCapacity=min_capacity,
        MaxCapacity=max_capacity
    )
except Exception as e:
    print(f"Error registering scalable target: {e}")
    response = None

{'ScalableTargetARN': 'arn:aws:application-autoscaling:us-east-1:010928219351:scalable-target/0d261df2de8558ed44809f9ba7e892bc09c8', 'ResponseMetadata': {'RequestId': '58bd46e4-c44c-4d13-b5b9-5376fdffc9b9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '58bd46e4-c44c-4d13-b5b9-5376fdffc9b9', 'content-type': 'application/x-amz-json-1.1', 'content-length': '131', 'date': 'Sun, 04 Aug 2024 17:58:10 GMT'}, 'RetryAttempts': 0}}


In [ ]:
response

También puede registrar un destino escalable en un índice secundario global. Por ejemplo, para un índice secundario global (“test-index”), el ID de recurso y los argumentos de dimensión escalable se actualizan adecuadamente.

In [ ]:
resource_id = "table/TestTable/index/test-index"
scalable_dimension = "dynamodb:index:WriteCapacityUnits"

### Crear una política de escalado

En este paso, se crea una política de escalado para TestTable. La política define los detalles según los cuales Auto Scaling de aplicaciones puede ajustar el rendimiento aprovisionado de la tabla y las acciones llevará a cabo para ello. Puede asociar esta política al objetivo escalable definido en el paso anterior (unidades de capacidad de escritura para la tabla TestTable).

La política contiene los componentes siguientes:

* **PredefinedMetricSpecification**: métrica que puede ajustar Auto Scaling de aplicaciones. Para DynamoDB, los siguientes valores son válidos para PredefinedMetricType:

    * DynamoDBReadCapacityUtilization

    * DynamoDBWriteCapacityUtilization

* **ScaleOutCooldown**: cantidad mínima de tiempo (en segundos) entre cada evento de Auto Scaling de aplicaciones que aumenta el rendimiento aprovisionado. Este parámetro permite que Auto Scaling de aplicaciones aumente de forma continua, pero no drástica, el rendimiento en respuesta a las cargas de trabajo reales. El ajuste predeterminado de ScaleOutCooldown es 0.

* **ScaleInCooldown**: cantidad mínima de tiempo (en segundos) entre cada evento de Auto Scaling de aplicaciones que reduce el rendimiento aprovisionado. Este parámetro permite que Auto Scaling de aplicaciones disminuya el rendimiento de manera gradual y predecible. El ajuste predeterminado de ScaleInCooldown es 0.

* **TargetValue**: Auto Scaling de aplicaciones se asegura de que la proporción entre capacidad consumida y capacidad aprovisionada se mantenga en este valor o en un valor próximo. TargetValue se define como un porcentaje.

Para entender cómo funciona TargetValue, imagine que tiene una tabla con una configuración de rendimiento aprovisionado de 200 unidades de capacidad de escritura. Decide crear una política de escalado para esta tabla, con un valor de TargetValue del 70 %.

Ahora, supongamos que comienza a dirigir el tráfico de escritura a la tabla, de tal forma que el rendimiento de escritura real es de 150 unidades de capacidad. La proporción entre capacidad consumida y aprovisionada es ahora de (150/200), es decir, del 75 %. Esta proporción supera su objetivo, de modo que Auto Scaling de aplicaciones aumenta la capacidad de escritura aprovisionada a 215 para que la proporción sea de (150/215), es decir, del 69,77 %; de esta forma se mantiene lo más próxima posible al valor de TargetValue, pero sin superarlo.

In [14]:
policy_config = {
    "PredefinedMetricSpecification": {
        "PredefinedMetricType": "DynamoDBWriteCapacityUtilization"
    },
    "ScaleOutCooldown": 60,
    "ScaleInCooldown": 60,
    "TargetValue": 50.0
}
policy_name = "MyScalingPolicy"
policy_type = "TargetTrackingScaling" 

In [15]:
try:
    response = client.put_scaling_policy(
        ServiceNamespace=service_namespace,
        ResourceId=resource_id,
        ScalableDimension=scalable_dimension,
        PolicyName=policy_name,
        PolicyType=policy_type,
        TargetTrackingScalingPolicyConfiguration=policy_config
    )
except Exception as e:
    print(f"Error putting scaling policy: {e}")
    response = None

{'PolicyARN': 'arn:aws:autoscaling:us-east-1:010928219351:scalingPolicy:1df2de85-58ed-4480-9f9b-a7e892bc09c8:resource/dynamodb/table/TestTable:policyName/MyScalingPolicy', 'Alarms': [{'AlarmName': 'TargetTracking-table/TestTable-AlarmHigh-99edb256-d32c-4c9e-8824-e95144bb70eb', 'AlarmARN': 'arn:aws:cloudwatch:us-east-1:010928219351:alarm:TargetTracking-table/TestTable-AlarmHigh-99edb256-d32c-4c9e-8824-e95144bb70eb'}, {'AlarmName': 'TargetTracking-table/TestTable-AlarmLow-d91052c3-52a0-4ecd-8d8a-c8858a54b51c', 'AlarmARN': 'arn:aws:cloudwatch:us-east-1:010928219351:alarm:TargetTracking-table/TestTable-AlarmLow-d91052c3-52a0-4ecd-8d8a-c8858a54b51c'}, {'AlarmName': 'TargetTracking-table/TestTable-ProvisionedCapacityHigh-38bd3347-e7c8-4ffb-aa76-6a18100607b9', 'AlarmARN': 'arn:aws:cloudwatch:us-east-1:010928219351:alarm:TargetTracking-table/TestTable-ProvisionedCapacityHigh-38bd3347-e7c8-4ffb-aa76-6a18100607b9'}, {'AlarmName': 'TargetTracking-table/TestTable-ProvisionedCapacityLow-ef938261-f3

In [16]:
response

{'PolicyARN': 'arn:aws:autoscaling:us-east-1:010928219351:scalingPolicy:1df2de85-58ed-4480-9f9b-a7e892bc09c8:resource/dynamodb/table/TestTable:policyName/MyScalingPolicy',
 'Alarms': [{'AlarmName': 'TargetTracking-table/TestTable-AlarmHigh-99edb256-d32c-4c9e-8824-e95144bb70eb',
   'AlarmARN': 'arn:aws:cloudwatch:us-east-1:010928219351:alarm:TargetTracking-table/TestTable-AlarmHigh-99edb256-d32c-4c9e-8824-e95144bb70eb'},
  {'AlarmName': 'TargetTracking-table/TestTable-AlarmLow-d91052c3-52a0-4ecd-8d8a-c8858a54b51c',
   'AlarmARN': 'arn:aws:cloudwatch:us-east-1:010928219351:alarm:TargetTracking-table/TestTable-AlarmLow-d91052c3-52a0-4ecd-8d8a-c8858a54b51c'},
  {'AlarmName': 'TargetTracking-table/TestTable-ProvisionedCapacityHigh-38bd3347-e7c8-4ffb-aa76-6a18100607b9',
   'AlarmARN': 'arn:aws:cloudwatch:us-east-1:010928219351:alarm:TargetTracking-table/TestTable-ProvisionedCapacityHigh-38bd3347-e7c8-4ffb-aa76-6a18100607b9'},
  {'AlarmName': 'TargetTracking-table/TestTable-ProvisionedCapacit

En el resultado, observe que Auto Scaling de aplicaciones ha creado dos alarmas de Amazon CloudWatch, una para cada límite (superior e inferior) del rango de escalado objetivo.

In [21]:
client.describe_scaling_policies(
    ServiceNamespace=service_namespace,
    ResourceId=resource_id
)

{'ScalingPolicies': [{'PolicyARN': 'arn:aws:autoscaling:us-east-1:010928219351:scalingPolicy:1df2de85-58ed-4480-9f9b-a7e892bc09c8:resource/dynamodb/table/TestTable:policyName/MyScalingPolicy',
   'PolicyName': 'MyScalingPolicy',
   'ServiceNamespace': 'dynamodb',
   'ResourceId': 'table/TestTable',
   'ScalableDimension': 'dynamodb:table:WriteCapacityUnits',
   'PolicyType': 'TargetTrackingScaling',
   'TargetTrackingScalingPolicyConfiguration': {'TargetValue': 50.0,
    'PredefinedMetricSpecification': {'PredefinedMetricType': 'DynamoDBWriteCapacityUtilization'},
    'ScaleOutCooldown': 60,
    'ScaleInCooldown': 60},
   'Alarms': [{'AlarmName': 'TargetTracking-table/TestTable-AlarmHigh-99edb256-d32c-4c9e-8824-e95144bb70eb',
     'AlarmARN': 'arn:aws:cloudwatch:us-east-1:010928219351:alarm:TargetTracking-table/TestTable-AlarmHigh-99edb256-d32c-4c9e-8824-e95144bb70eb'},
    {'AlarmName': 'TargetTracking-table/TestTable-AlarmLow-d91052c3-52a0-4ecd-8d8a-c8858a54b51c',
     'AlarmARN': 'a

### Dirigir tráfico de escritura a TestTable

Ahora puede probar la política de escalado escribiendo datos en TestTable.

In [33]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table("TestTable")
filler = "x" * 100000

In [ ]:
i = 0
while (i < 10):
    j = 0
    while (j < 10):
        
        table.put_item(
            Item={
                'PK':str(i),
                'SK':str(j),
                'filler':{"S":filler}
            }
        )
        j += 1
    i += 1
    print (i, j)

In [25]:
client.describe_scaling_activities(
    ServiceNamespace=service_namespace,
    ResourceId=resource_id
)

{'ScalingActivities': [{'ActivityId': '6176ced7-45bd-4f2a-8382-f79d3cee429e',
   'ServiceNamespace': 'dynamodb',
   'ResourceId': 'table/TestTable',
   'ScalableDimension': 'dynamodb:table:WriteCapacityUnits',
   'Description': 'Setting write capacity units to 10.',
   'Cause': 'monitor alarm TargetTracking-table/TestTable-AlarmHigh-99edb256-d32c-4c9e-8824-e95144bb70eb in state ALARM triggered policy MyScalingPolicy',
   'StartTime': datetime.datetime(2024, 8, 4, 15, 14, 38, 930000, tzinfo=tzlocal()),
   'EndTime': datetime.datetime(2024, 8, 4, 15, 15, 15, 214000, tzinfo=tzlocal()),
   'StatusCode': 'Successful',
   'StatusMessage': 'Successfully set write capacity units to 10. Change successfully fulfilled by dynamodb.'}],
 'ResponseMetadata': {'RequestId': '9c0d2a02-8ef8-479d-8300-04f9d8da3b1f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9c0d2a02-8ef8-479d-8300-04f9d8da3b1f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '583',
   'date': 

Esto indica que la aplicación de Auto Scaling ha emitido una solicitud UpdateTable a DynamoDB.

In [31]:
table_data = dt.table.meta.client.describe_table(TableName=dt.table_name)
table_data['Table']['ProvisionedThroughput']

{'LastIncreaseDateTime': datetime.datetime(2024, 8, 4, 15, 14, 40, 461000, tzinfo=tzlocal()),
 'NumberOfDecreasesToday': 0,
 'ReadCapacityUnits': 5,
 'WriteCapacityUnits': 10}

WriteCapacityUnits se ha escalado de 5 a 10.